In [0]:
dbutils.widgets.dropdown(name = 'environment',choices=['dev', 'uat', 'prod'], defaultValue='dev', label='select the environment')
environment = dbutils.widgets.get('environment')    

In [0]:
catalog_url = spark.sql(f"DESCRIBE EXTERNAL LOCATION traffic_{environment}").select('url').collect()[0][0]
catalog_url

In [0]:
landing_url = f"{catalog_url}landing/"
silver_url = f"{catalog_url}medallion/silver/"
gold_url = f"{catalog_url}medallion/gold/"
bronze_url = f"{catalog_url}medallion/bronze/"
checkpoint_url = f"{catalog_url}checkpoints/"
db_name = "traffic_db"

In [0]:
def cleanup():
    if spark.sql(f"SHOW DATABASES IN traffic_{environment}_catalog").filter(f"databaseName == '{db_name}'").count() == 1:
        print(f"Dropping the database traffic_{environment}_catalog.{db_name}...", end='')
        spark.sql(f"DROP DATABASE traffic_{environment}_catalog.{db_name} CASCADE")
        print("Done")
    print(f"Deleting {checkpoint_url}...", end='')
    dbutils.fs.rm(checkpoint_url, True)
    dbutils.fs.mkdirs(checkpoint_url)
    print("Done")
    print(f"Deleting {bronze_url}...", end='')
    dbutils.fs.rm(bronze_url, True)
    dbutils.fs.mkdirs(bronze_url)
    print("Done")
    print(f"Deleting {silver_url}...", end='')
    dbutils.fs.rm(silver_url, True)
    dbutils.fs.mkdirs(silver_url)
    print("Done")
    print(f"Deleting {gold_url}...", end='')
    dbutils.fs.rm(gold_url, True)
    dbutils.fs.mkdirs(gold_url)
    print("Done")

In [0]:
cleanup()

In [0]:
def create_db():
    #spark.catalog.clearCache()
    print(f"Creating the database traffic_{environment}_catalog.{db_name}...", end='')
    spark.sql(f"CREATE DATABASE IF NOT EXISTS traffic_{environment}_catalog.{db_name}")
    spark.sql(f"USE traffic_{environment}_catalog.{db_name}")
    print("Done")

In [0]:
def createTable_rawTraffic(environment):
    print(f'Creating raw_Traffic table in traffic_{environment}_catalog')
    spark.sql(f"""CREATE TABLE IF NOT EXISTS `traffic_{environment}_catalog`.`{db_name}`.`raw_traffic`
                        (
                            Record_ID INT,
                            Count_point_id INT,
                            Direction_of_travel VARCHAR(255),
                            Year INT,
                            Count_date VARCHAR(255),
                            hour INT,
                            Region_id INT,
                            Region_name VARCHAR(255),
                            Local_authority_name VARCHAR(255),
                            Road_name VARCHAR(255),
                            Road_Category_ID INT,
                            Start_junction_road_name VARCHAR(255),
                            End_junction_road_name VARCHAR(255),
                            Latitude DOUBLE,
                            Longitude DOUBLE,
                            Link_length_km DOUBLE,
                            Pedal_cycles INT,
                            Two_wheeled_motor_vehicles INT,
                            Cars_and_taxis INT,
                            Buses_and_coaches INT,
                            LGV_Type INT,
                            HGV_Type INT,
                            EV_Car INT,
                            EV_Bike INT,
                            Extract_Time TIMESTAMP
                    )
                USING DELTA
                LOCATION '{bronze_url}raw_traffic/'
            """)
    print("Done")
                    
  


In [0]:
def createTable_rawRoads(environment):
    print(f'Creating raw_Roads table in traffic_{environment}_catalog')
    spark.sql(f"""CREATE TABLE IF NOT EXISTS `traffic_{environment}_catalog`.`{db_name}`.`raw_roads`
                        (
                            Road_ID INT,
                            Road_Category_Id INT,
                            Road_Category VARCHAR(255),
                            Region_ID INT,
                            Region_Name VARCHAR(255),
                            Total_Link_Length_Km DOUBLE,
                            Total_Link_Length_Miles DOUBLE,
                            All_Motor_Vehicles DOUBLE
                    )
                USING DELTA
                LOCATION '{bronze_url}raw_roads/'
            """)
    print("Done")

In [0]:
create_db()
createTable_rawTraffic(environment)
createTable_rawRoads(environment)